In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import plotly.express as px
import datetime
from scipy import stats
import requests
import zipfile, io

# Examining the Effects of a COVID Surge on Depression
In this notebook, we will look at the causal effects of a COVID surge on the probability of experiencing depression symptoms. 

We want to identify a time period in which one region of the country went through a large COVID surge while the other did not. Experiencing a surge will, in effect, be the treatment, while the other region that does not experience the surge will serve as a control group. We will ultimately employ a difference-in-differences design to isolate the treatment effect.

## COVID Death Trends
We start by examining COVID mortality rates using the provided dataset.

In [2]:
df = pd.read_csv("data/time_series_covid19_deaths_US.csv")
df.tail()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/11/22,4/12/22,4/13/22,4/14/22,4/15/22,4/16/22,4/17/22,4/18/22,4/19/22,4/20/22
3337,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,16,16,16,16,16,16,16,16,16,16
3338,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,39,39,39,39,39,39,39,39,39,39
3339,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,43,43,43,43,43,43,43,43,44,44
3341,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,...,18,18,18,18,18,18,18,18,18,18


This dataset has counts of new deaths for each day, disaggregated by state. We start by grouping the data by state and adding the counts up so that we see cumulative deaths by day for each state:

In [3]:
df_time = (df
           .groupby("Province_State")
           .sum()
           .iloc[:, 12:]
           .T)
df_time.tail()

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
4/16/22,19502,1235,18,29823,11339,89670,12023,10813,2894,0,...,26109,87809,4736,623,111,20022,12619,6794,14390,1801
4/17/22,19502,1235,18,29823,11348,89670,12023,10813,2894,0,...,26109,87809,4736,623,111,20022,12619,6794,14390,1801
4/18/22,19510,1235,18,29823,11354,89697,12029,10813,2894,0,...,26109,87849,4736,623,111,20044,12626,6796,14391,1801
4/19/22,19513,1235,21,29823,11354,89797,12036,10813,2896,0,...,26110,87892,4736,626,111,20071,12626,6804,14395,1807
4/20/22,19524,1248,21,29852,11360,89838,12030,10825,2896,0,...,26109,87929,4736,626,111,20099,12650,6807,14399,1807


We proceed to revert to having daily new deaths in our new dataset structure, where the index is the date and each column is a state/other location (e.g. Diamond Princess, which is a cruise ship)

In [4]:
df_time_delta = (df_time
                 .diff()
                 .fillna(0)
                 .iloc[1:, :])
df_time_delta.tail()

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
4/16/22,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4/17/22,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4/18/22,8.0,0.0,0.0,0.0,6.0,27.0,6.0,0.0,0.0,0.0,...,0.0,40.0,0.0,0.0,0.0,22.0,7.0,2.0,1.0,0.0
4/19/22,3.0,0.0,3.0,0.0,0.0,100.0,7.0,0.0,2.0,0.0,...,1.0,43.0,0.0,3.0,0.0,27.0,0.0,8.0,4.0,6.0
4/20/22,11.0,13.0,0.0,29.0,6.0,41.0,-6.0,12.0,0.0,0.0,...,-1.0,37.0,0.0,0.0,0.0,28.0,24.0,3.0,4.0,0.0


We next designate regions of interest - the Southeast and the Northeast. We chose these regions based on prior knowledge that these two regions have typically had a degree of inverse correlation in COVID death rates.

In [5]:
SE_states = ["Alabama", "Florida", "Georgia", "Arkansas", "Kentucky",
             "Louisiana", "Mississippi", "North Carolina", "South Carolina", "Tennessee"]
NE_states = ["Maine", "New York", "New Jersey", "Delaware", "Vermont",
             "Massachusetts", "Rhode Island", "Connecticut", "New Hampshire", "Pennsylvania"]

In [6]:
average_southeast = df_time_delta[SE_states].sum(axis=1)
average_northeast = df_time_delta[NE_states].sum(axis=1)

In [7]:
south_v_north = (pd
                 .concat([average_northeast, average_southeast],
                         axis=1)
                 .rename(columns={0: "Northeast",
                                  1: "Southeast"})
                )

We proceed to visualize basic trends - a line plot of daily death counts confirms that there are various periods of inverse correlation between the two regions

In [187]:
roll = 14
fig = px.line(south_v_north.rolling(roll).mean(),
              title=f"Southeast vs. Northeast, Daily Deaths Count, {roll} Day Rolling Average"
              )
fig.show()

We next incorporate population data for each state to convert our figures to deaths per capita to confirm the trend

In [188]:
base_url = "https://www2.census.gov/programs-surveys/popest"
population = pd.read_excel(base_url + "/tables/2020-2021/state/totals/NST-EST2021-POP.xlsx", skiprows=2)

In [189]:
population = population.iloc[6:57, :]

In [190]:
population.loc[:, "State"] = (population
                              .loc[:, "Geographic Area"]
                              .str
                              .strip(".")
                             )
population_isolated = population[["State", "April 1, 2020 Estimates Base"]]
population_isolated = population_isolated.set_index("State")

In [191]:
per_capita = df_time_delta[population_isolated.T.columns].div(population_isolated.T.iloc[0, :], axis=1) * 100

In [192]:
average_southeast_pc = per_capita[SE_states].sum(axis=1)
average_northeast_pc = per_capita[NE_states].sum(axis=1)
south_v_north_pc = (pd
                    .concat([average_northeast_pc, average_southeast_pc], axis=1)
                    .rename(columns={0: "Northeast", 1: "Southeast"})
                   )

In [193]:
roll = 14
fig = px.line(south_v_north_pc.rolling(roll).mean(),
              title=f"Southeast vs. Northeast, Deaths per 100 people, {roll} Day Rolling Average"); 
fig.show()

We are especially interested in the time period between 4/21/2021 and 10/30/2021, where the two regions start with parallel trends, then diverge, where the southeast has a very large COVID surge and the northeast does not.  

The same point in time can be visualized in terms of correlation below, where the regions start by being very correlated, peaking at 5/31/2021 then diverge.

In [186]:
days = 14
fig = px.line(average_southeast.rolling(days).corr(average_northeast_pc),
              title=f"Southeast vs. Northeast, Correlation Coefficient over past {days} Days"); 
fig.show()

### Retrieving Census Pulse Data

In [16]:
from sklearn.preprocessing import OneHotEncoder
from os.path import exists

## 1.) Loading Census Pulse Data

In [98]:
#Household Pulse Survey PUF: July 21 – August 2
zip_url = 'https://www2.census.gov/programs-surveys/demo/datasets/hhp/2021/wk34/HPS_Week34_PUF_CSV.zip'

if not exists("data/pulse_1.csv"):
    print("Retreiving url")
    response = requests.get(zip_url)
    print("Unzipping")
    zf = zipfile.ZipFile(io.BytesIO(response.content))
    print("Reading into pandas")
    pulse_1 = pd.read_csv(zf.open('pulse2021_puf_34.csv'))
    del zf
    print("Saving")
    pulse_1.to_csv("data/pulse_1.csv")
else:
    print("Opening local file for pulse 1 - wk 34 July 21 – August 2")
    pulse_1 = pd.read_csv("data/pulse_1.csv")

Opening local file for pulse 1 - wk 34 July 21 – August 2


In [99]:
#Household Pulse Survey PUF: September 15 - September 27
zip_url = 'https://www2.census.gov/programs-surveys/demo/datasets/hhp/2021/wk38/HPS_Week38_PUF_CSV.zip'

if not exists("data/pulse_2.csv"):
    print("Retreiving url")
    response = requests.get(zip_url)
    print("Unzipping")
    zf = zipfile.ZipFile(io.BytesIO(response.content))
    print("Reading into pandas")
    pulse_2 = pd.read_csv(zf.open('pulse2021_puf_38.csv'))
    del zf
    print("Saving")
    pulse_2.to_csv("data/pulse_2.csv")
else:
    print("Opening local file for pulse 2 wk 38 September 15 - September 27")
    pulse_2 = pd.read_csv("data/pulse_2.csv")

Opening local file for pulse 2 wk 38 September 15 - September 27


In [100]:
pulse_1["Pulse_2"] = 0
pulse_2["Pulse_2"] = 1

In [101]:
pulse_1 = pd.concat([pulse_1, pulse_2])

## 2.) Isolating To Regions of Interest
We want to examine only the Northeast and the Southeast using our previous definition retrieved from Wikipedia, so wec reate a dataframe `state_codes` that maps state names to FIPS codes, `st_code` 

In [102]:
df["st_code"] = df["FIPS"].astype(str).apply(lambda x: "0" + x if len(x) < 7 else x).str[:2]
state_codes = df[["st_code", "Province_State"]].drop_duplicates(subset="Province_State")
state_codes.head()

,st_code,Province_State
0,01,Alabama
69,02,Alaska
102,06,American Samoa
103,04,Arizona
120,05,Arkansas


In [103]:
SE_codes = state_codes[state_codes["Province_State"].isin(SE_states)]["st_code"].astype(int)
NE_codes = state_codes[state_codes["Province_State"].isin(NE_states)]["st_code"].astype(int)

In [104]:
# restrict data to only those in the north or south east
pulse_1_in_region = pulse_1[pulse_1["EST_ST"].isin(pd.concat([SE_codes, NE_codes], axis=0))]
pulse_1_in_region = pulse_1_in_region.replace({-88: None, -99: None})

We proceed to add a dummy variable, `pulse_1_in_region["NORTHEAST"]`, which marks $1$ if the respondent is in the northeast, and $0$ if they are in the southeast

In [105]:
pulse_1_in_region["NORTHEAST"] = pulse_1_in_region["EST_ST"].isin(NE_codes).astype(int)
pulse_1_in_region[["EST_ST", "NORTHEAST"]].sample(5, random_state=4)

,EST_ST,NORTHEAST
42259,12,0
40628,45,0
52197,42,1
52792,12,0
30144,34,1


In [106]:
pulse_1_in_region["DID"] = pulse_1_in_region["NORTHEAST"] * pulse_1_in_region["Pulse_2"]

## 3.) Manual Feature Engineering

In [107]:
pulse_1_in_region["AGE"] = 2022 - pulse_1_in_region["TBIRTH_YEAR"]
pulse_1_in_region["AGE^2"] = pulse_1_in_region["AGE"]**2

In [108]:
pulse_1_in_region["RACE"] = (
    (pulse_1_in_region["RRACE"].astype(str) +
     pulse_1_in_region["RHISPANIC"].astype(str))
    .map({'11': "White Non Hispanic",
          '12': "White Hispanic",
          "21": "Black Non Hispanic",
          "22": "Black Hispanic",
          "31": "Asian Non Hispanic",
          "32": "Asian Hispanic",
          "41": "Other Non Hispanic",
          "42": "Other, Hispanic"}))

In [109]:
pulse_1_in_region["MS"] = (pulse_1_in_region["MS"]
                           .map({1: "Married",
                                 2: "Widow",
                                 3: "Divorce",
                                 4: "Separated",
                                 5: "Single"}))

In [110]:
pulse_1_in_region["LGBTQ"] = pulse_1_in_region["SEXUAL_ORIENTATION"].isin([1, 3]).astype(int)

In [111]:
pulse_1_in_region["HAS_INS"] = (
    ((pulse_1_in_region[["HLTHINS1",
                         "HLTHINS2",
                         "HLTHINS3",
                         "HLTHINS4",
                         "HLTHINS5",
                         "HLTHINS6",
                         "HLTHINS7",
                         "HLTHINS8"]] != 2)
     .astype(int)
     .sum(axis=1) > 0)
    .astype(int)
)

In [112]:
pulse_1_in_region["EEDUC"] = (
    pulse_1_in_region["EEDUC"].map(
        {1: "1_Less than high school",
         2: "2_Some high school",
         3: "3_High school graduate or equivalent",
         4: "4_Some college, but degree not received or is in progress",
         5: "5_Associate’s degree",
         6: "6_Bachelor's degree",
         7: "7_Graduate degree",
         }))

In [113]:
pulse_1_in_region["WRKLOSSRV"] = (pulse_1_in_region["WRKLOSSRV"] == 1).astype(int)
pulse_1_in_region["CTC_YN"] = (pulse_1_in_region["CTC_YN"] == 1).astype(int)
pulse_1_in_region["WORK_ONSITE"] = (pulse_1_in_region["ACTIVITY1"] == 1).astype(int)
pulse_1_in_region["SNAP"] = (pulse_1_in_region["SNAP_YN"] == 1).astype(int)
pulse_1_in_region["BEHIND_ON_RENT"] = (pulse_1_in_region["TENURE"] == 4).astype(int)
pulse_1_in_region["FEMALE"] = (pulse_1_in_region["GENID_DESCRIBE"] == 2).astype(int)

Note, including Income will lower the sample size too much.

In [114]:
vars_of_interest = ["ANXIOUS",
                    "DOWN",
                    "HAS_INS",
                    "AGE",
                    "AGE^2",
                    "LGBTQ",
                    "WRKLOSSRV",
                    "CTC_YN",
                    "WORK_ONSITE",
                    "SNAP",
                    "BEHIND_ON_RENT",
                    "FEMALE",
                    "Pulse_2",
                    "DID",
                    # To be Encoded:
                    "RACE",
                    "EEDUC",
                    "MS",
                    "EXPNS_DIF",
                    "CURFOODSUF",
                   "NORTHEAST"]
pulse_1_vars = pulse_1_in_region.dropna(
    subset=vars_of_interest)[vars_of_interest]
pulse_1_vars = pulse_1_vars.reset_index().drop(columns="index")
pulse_1_vars.head()

,ANXIOUS,DOWN,HAS_INS,AGE,AGE^2,LGBTQ,WRKLOSSRV,CTC_YN,WORK_ONSITE,SNAP,BEHIND_ON_RENT,FEMALE,Pulse_2,DID,RACE,EEDUC,MS,EXPNS_DIF,CURFOODSUF,NORTHEAST
0,1.0,1.0,1,36,1296,0,0,1,1,0,0,1,0,0,White Non Hispanic,7_Graduate degree,Married,1.0,1.0,0
1,4.0,1.0,1,81,6561,0,0,0,0,0,0,1,0,0,White Non Hispanic,"4_Some college, but degree not received or is ...",Widow,1.0,1.0,0
2,4.0,4.0,1,60,3600,0,0,0,0,0,0,0,0,0,White Non Hispanic,7_Graduate degree,Married,1.0,1.0,0
3,3.0,3.0,1,37,1369,1,0,0,1,1,0,1,0,0,White Non Hispanic,"4_Some college, but degree not received or is ...",Divorce,2.0,1.0,0
4,3.0,4.0,1,43,1849,0,0,1,1,1,0,1,0,0,White Non Hispanic,1_Less than high school,Married,4.0,2.0,0


In [139]:
def process_pulse(data):
    manual_vars = 12
    X_frame = data.iloc[:, manual_vars:]
    enc = OneHotEncoder(drop='first')
    enc.fit(X_frame)
    X = pd.DataFrame(enc.transform(X_frame).toarray(),
                     columns=enc.get_feature_names(X_frame.columns))
    X = pd.concat([data[vars_of_interest[2:(manual_vars)]], X], axis=1)
    y = ((data["DOWN"] == 4) | (data["ANXIOUS"] == 4)).astype(int)
    return X, y

In [179]:
X, y = process_pulse(pulse_1_vars)
class_bal = round((y.value_counts(normalize=True)*100), 8).to_dict()
class_bal

{0: 85.87043529, 1: 14.12956471}

In [141]:
X.head()

,HAS_INS,AGE,AGE^2,LGBTQ,WRKLOSSRV,CTC_YN,WORK_ONSITE,SNAP,BEHIND_ON_RENT,FEMALE,...,MS_Separated,MS_Single,MS_Widow,EXPNS_DIF_2.0,EXPNS_DIF_3.0,EXPNS_DIF_4.0,CURFOODSUF_2.0,CURFOODSUF_3.0,CURFOODSUF_4.0,NORTHEAST_1
0,1,36,1296,0,0,1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,81,6561,0,0,0,0,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,60,3600,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,37,1369,1,0,0,1,1,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,43,1849,0,0,1,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [142]:
y

0        0
1        1
2        1
3        0
4        1
        ..
36224    0
36225    0
36226    0
36227    0
36228    0
Length: 36229, dtype: int32

## Modeling:

In [143]:
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [144]:
def class_metrics(pred, y):
    train_accuracy = np.mean(pred == y)
    precision = np.sum((pred == y) & (pred == 1)) / np.sum(pred)
    recall = np.sum((pred == y) & (pred == 1)) / np.sum(y)
    print(f"accuracy: {train_accuracy:.4f}")
    print(f'precision = {precision:.4f}')
    print(f'recall = {recall:.4f}')
    return train_accuracy, precision, recall


def regression_results(y_true, y_pred):
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

## Linear Probability Model

In [146]:
lm_prob = LinearRegression(
        fit_intercept=True)
lm_prob.fit(X_train, y_train)

LinearRegression()

We start by training a Linear Regression model on the data. In this case, because our dependent variable is binomial random variable, our model can be called a Linear Probability model. This model specification caries a number of issues - namely, even though the dependent variable is a probability bounded 0 to 1 inclusive, our model can predict values greater than 1 and less than 0, which will lead our predictions to be biased, and will render the parametric standard errors of our coefficients unusable. However, it is still possible to interpret our coefficient estimates as average marginal effects on the probability of being depressed, and we will bootstrap standard errors for confidence intervals.
Sources:  
- https://opa.hhs.gov/sites/default/files/2020-07/lpm-tabrief.pdf
- https://murraylax.org/rtutorials/linearprob.html#content
- https://www.econometrics-with-r.org/11-1-binary-dependent-variables-and-the-linear-probability-model.html


##### Training Stats

In [147]:
regression_results(y_train, lm_prob.predict(X_train))

explained_variance:  0.1709
r2:  0.1709
MAE:  0.2038
MSE:  0.1008
RMSE:  0.3174


In [148]:
pd.Series(lm_prob.predict(X_train)).describe()

count    28983.000000
mean         0.141566
std          0.144098
min         -0.097733
25%          0.043027
50%          0.098147
75%          0.191028
max          0.878135
dtype: float64

In [149]:
theshold = .5
lm_prob_predictions = np.array([1 if x > theshold else 0 for x in lm_prob.predict(X_train)])
class_metrics(lm_prob_predictions, y_train);

accuracy: 0.8663
precision = 0.6048
recall = 0.1611


##### Test Stats

In [150]:
regression_results(y_test, lm_prob.predict(X_test))

explained_variance:  0.1639
r2:  0.1638
MAE:  0.2037
MSE:  0.1008
RMSE:  0.3175


In [151]:
pd.Series(lm_prob.predict(X_test)).describe()

count    7246.000000
mean        0.141535
std         0.144434
min        -0.103338
25%         0.041531
50%         0.100560
75%         0.191098
max         0.875010
dtype: float64

In [152]:
lm_prob_predictions_test = np.array([1 if x > theshold else 0 for x in lm_prob.predict(X_test)])
class_metrics(lm_prob_predictions_test, y_test);

accuracy: 0.8685
precision = 0.6137
recall = 0.1673


We run the same model specification in `statsmodels` in order to get a look at the coefficients, though we will not be examining the standard errors, t-scores, or p-values given the limitations of the linear probability model.

In [153]:
model_lm_prob = sm.OLS(y_train,sm.add_constant(X_train))
results_lm_prob = model_lm_prob.fit()
results_lm_prob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     165.7
Date:                Wed, 27 Apr 2022   Prob (F-statistic):               0.00
Time:                        17:04:06   Log-Likelihood:                -7867.1
No. Observations:               28983   AIC:                         1.581e+04
Df Residuals:                   28946   BIC:                         1.611e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                               0.1944      0.055      3.506      0.000       0.086       0.303
HAS_INS                                                            -0.0206      0.010     -2.118      0.034      -0.040      -0.002
AGE                                                                -0.0048      0.001     -5.639      0.000      -0.006      -0.003
AGE^2                                                            1.836e-05   7.67e-06      2.393      0.017    3.32e-06    3.34e-05
LGBTQ                                                               0.0662      0.008      8.502      0.000       0.051       0.081
WRKLOSSRV                                                           0.0332      0.006      5.314      0.000       0.021       0.045
CTC_YN                                                             -0.0049      0.005     -0.927      0.354      -0.015       0.005
WORK_ONSITE                                                        -0.0086      0.004     -2.063      0.039      -0.017      -0.000
SNAP                                                                0.0290      0.008      3.832      0.000       0.014       0.044
BEHIND_ON_RENT                                                      0.0206      0.017      1.239      0.215      -0.012       0.053
FEMALE                                                              0.0226      0.004      5.814      0.000       0.015       0.030
Pulse_2_1                                                           0.0057      0.005      1.089      0.276      -0.005       0.016
DID_1                                                               0.0039      0.007      0.526      0.599      -0.011       0.019
RACE_Asian Non Hispanic                                             0.0396      0.044      0.905      0.365      -0.046       0.125
RACE_Black Hispanic                                                 0.0074      0.048      0.156      0.876      -0.086       0.101
RACE_Black Non Hispanic                                             0.0261      0.043      0.608      0.543      -0.058       0.110
RACE_Other Non Hispanic                                             0.1212      0.044      2.735      0.006       0.034       0.208
RACE_Other, Hispanic                                                0.1085      0.048      2.254      0.024       0.014       0.203
RACE_White Hispanic                                                 0.0593      0.043      1.372      0.170      -0.025       0.144
RACE_White Non Hispanic                                             0.0999      0.043      2.346      0.019       0.016       0.183
EEDUC_2_Some high school                       

In [154]:
models = []
for ind in range(1000):
    data = pd.concat([y, X], axis=1).sample(X.shape[0], replace=True, random_state=None)
    y_, X_ = data.iloc[:, 0], data.iloc[:, 1:]
    lm_prob_ = LinearRegression(fit_intercept=True).fit(X_, y_)
    models.append(lm_prob_)
    if ind % 100 == 0:
        print(ind, "Done")

0 Done
100 Done
200 Done
300 Done
400 Done
500 Done
600 Done
700 Done
800 Done
900 Done


In [183]:
confs = pd.DataFrame([model.coef_ for model in models])
results = pd.concat([confs.quantile(.025), confs.quantile(.5), confs.quantile(.975)], axis=1).T
results.columns = X.columns
results = results.T
results["Significant"] = -((results[0.025] < 0) & (results[0.975] > 0))
results.sort_values(by=0.5, key=np.abs, ascending=False)

,0.025,0.5,0.975,Significant
EXPNS_DIF_4.0,0.290015,0.311125,0.333606,True
CURFOODSUF_4.0,0.227237,0.272674,0.320519,True
EXPNS_DIF_3.0,0.120673,0.135630,0.150128,True
"RACE_Other, Hispanic",0.032402,0.129604,0.210365,True
RACE_Other Non Hispanic,0.053062,0.128588,0.194437,True
CURFOODSUF_3.0,0.096085,0.122401,0.150733,True
RACE_White Non Hispanic,0.035447,0.107818,0.169627,True
LGBTQ,0.049743,0.067477,0.083513,True
RACE_White Hispanic,-0.009491,0.064426,0.127441,False
EXPNS_DIF_2.0,0.047542,0.057599,0.067201,True


## Logistic Regression Classifier

In [166]:
lr = LogisticRegression(fit_intercept=True, max_iter=10000)
lr.fit(X_train, y_train)

print("Train")
class_metrics(lr.predict(X_train), y_train)
print("Test")
class_metrics(lr.predict(X_test), y_test)

Train
accuracy: 0.8641
precision = 0.5594
recall = 0.1894
Test
accuracy: 0.8692
precision = 0.5983
recall = 0.2037


(0.8691691967982335, 0.5982658959537572, 0.20374015748031496)

In [162]:
model_logit = sm.Logit(y_train, sm.add_constant(X_train)).fit()
model_logit.summary()

Optimization terminated successfully.
         Current function value: 0.334751
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                28983
Model:                          Logit   Df Residuals:                    28946
Method:                           MLE   Df Model:                           36
Date:                Wed, 27 Apr 2022   Pseudo R-squ.:                  0.1791
Time:                        17:35:36   Log-Likelihood:                -9702.1
converged:                       True   LL-Null:                       -11819.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================================================
                                                                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
const                                                              -2.3771      0.527     -4.510      0.000      -3.410      -1.344
HAS_INS                                                            -0.1036      0.076     -1.358      0.174      -0.253       0.046
AGE                                                                -0.0041      0.009     -0.480      0.631      -0.021       0.013
AGE^2                                                              -0.0002   8.26e-05     -2.846      0.004      -0.000   -7.32e-05
LGBTQ                                                               0.4580      0.064      7.124      0.000       0.332       0.584
WRKLOSSRV                                                           0.2061      0.050      4.107      0.000       0.108       0.305
CTC_YN                                                             -0.0502      0.048     -1.040      0.298      -0.145       0.044
WORK_ONSITE                                                        -0.0800      0.041     -1.938      0.053      -0.161       0.001
SNAP                                                                0.1729      0.060      2.865      0.004       0.055       0.291
BEHIND_ON_RENT                                                      0.1584      0.137      1.156      0.248      -0.110       0.427
FEMALE                                                              0.2262      0.040      5.673      0.000       0.148       0.304
Pulse_2_1                                                           0.0540      0.051      1.062      0.288      -0.046       0.154
DID_1                                                               0.0489      0.075      0.655      0.513      -0.097       0.195
RACE_Asian Non Hispanic                                             0.1404      0.443      0.317      0.751      -0.727       1.008
RACE_Black Hispanic                                                 0.0469      0.470      0.100      0.921      -0.874       0.967
RACE_Black Non Hispanic                                             0.1404      0.430      0.326      0.744      -0.703       0.984
RACE_Other Non Hispanic                                             0.9093      0.439      2.072      0.038       0.049       1.769
RACE_Other, Hispanic                                                0.7655      0.463      1.652      0.099      -0.143       1.674
RACE_White Hispanic                                                 0.4360      0.433      1.008      0.314      -0.412       1.284
RACE_White Non Hispanic                                             0.7923      0.427      1.854      0.064      -0.045       1.630
EEDUC_2_Some high school                                           -0.1763      0.256     -0.688      0.492      -0.679       0.326
EEDUC_3_High school graduate or equivalent                         -0.336

In [163]:
X_1 = X.drop(columns=[
    "LGBTQ",
    'WORK_ONSITE',
    'RACE_Black Non Hispanic',
    'RACE_Other Non Hispanic',
    'RACE_White Non Hispanic',
    'CTC_YN',
    'EEDUC_4_Some college, but degree not received or is in progress',
    'MS_Separated',
    'MS_Single',
    'EEDUC_2_Some high school',
    'MS_Widow',
])
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    X_1, y.values, test_size=.2, random_state=42)

In [164]:
lr = LogisticRegression(
        fit_intercept=True, max_iter=1000)
lr.fit(X_train_1, y_train_1)

print("Train")
class_metrics(lr.predict(X_train_1), y_train_1)
print("Test")
class_metrics(lr.predict(X_test_1), y_test_1)

Train
accuracy: 0.8624
precision = 0.5453
recall = 0.1687
Test
accuracy: 0.8693
precision = 0.6068
recall = 0.1929


(0.8693072039746067, 0.6068111455108359, 0.19291338582677164)

### Bootstrapping

In [ ]:
models = []
for ind in range(100):
    # sample then check if minority class present
    data=pd.concat([y, X], axis=1).sample(X.shape[0], replace=True)
    y_, X_ = data.iloc[:, 0], data.iloc[:, 1:]
    lr = LogisticRegression(
        fit_intercept=True, max_iter=100, penalty="none")
    lr.fit(X_, y_)
    models.append(lr)
    if ind % 10 == 0:
        print(ind)

In [ ]:
confs = pd.DataFrame([model.coef_[0] for model in models])
results = pd.concat([confs.quantile(.025), confs.quantile(.5), confs.quantile(.975)], axis=1).T
results.columns = X.columns
results = results.T

In [ ]:
results["Significant"] = -((results[0.025] < 0) & (results[0.975] > 0))
results["$|\\beta_{i}|$"] = np.abs(results[0.5])

In [ ]:
results.sort_values(by="$|\\beta_{i}|$", ascending=False)